### Data Ops Challenge -  William Burke

In [1]:
# Importing libraries I might use
import pandas as pd
import numpy as np
import sqlite3

### Establishing and Exploring DB

In [2]:
# Connecting to the db
connection = sqlite3.connect("interview.db")
cursor = connection.cursor()

In [3]:
# Verify connection object (should print 0)
print("Changes:", connection.total_changes)

# Getting the names of tables
query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
tableNames = cursor.fetchall()
print("Table names:", tableNames)

Changes: 0
Table names: [('model_scores_by_zip',), ('roster_1',), ('roster_2',), ('roster_3',), ('roster_4',), ('roster_5',), ('std_member_info',)]


In [4]:
# Getting some summary info for all tables
for table in tableNames:
    table = table[0]
    print("***********\nPrinting info about table:", table)
    
    # Getting number of records
    query = "SELECT COUNT(*) FROM " + table
    result = cursor.execute(query)
    print("# Records",result.fetchone())

    # Getting attribute names and first record
    query = "SELECT * FROM " + table + " LIMIT 1"
    result = cursor.execute(query)
    print("Column names:", list(map(lambda x: x[0], result.description)))
    print("First record:", result.fetchone())

***********
Printing info about table: model_scores_by_zip
# Records (1760,)
Column names: ['zcta', 'state_code', 'state name', 'neighborhood_stress_score', 'algorex_sdoh_composite_score', 'social_isolation_score', 'transportation_access_score', 'food_access_score', 'unstable_housing_score', 'state_govt_assistance', 'homeless_indicator', 'derived_indicator']
First record: (94720, 6.0, 'California', -0.31, 6.19, 2.37, 3.89, 3.06, 2.03, 0.59, 10.0, 1)
***********
Printing info about table: roster_1
# Records (23659,)
Column names: ['Person_Id', 'First_Name', 'Last_Name', 'Dob', 'Age', 'Gender', 'Street_Address', 'State', 'City', 'Zip', 'eligibility_start_date', 'eligibility_end_date', 'payer']
First record: ('15340001', 'Daniel', 'Smith', '2017-04-27', '5', 'Male', '1505 Alvarez Spur Suite 902', 'California', 'Lake Sharonburgh', '93546', '2021-08-01', '2021-11-01', 'Madv')
***********
Printing info about table: roster_2
# Records (23392,)
Column names: ['Person_Id', 'First_Name', 'Last_N

In [5]:
# Inconsistencies:
# Dob is formatted differently in roster_2: NEED to fix
# State is abbreviated in roster_4: DOES NOT affect anything... state is not a FK and doesn't need to be queried in this challenge.
# There are also quite a few inconsistencies with the order of attributes: This matters when doing UNION of tables

In [6]:
# ***** Looking into dates *****
# Checking date stuff for roster_1
query = "SELECT * FROM roster_1 WHERE eligibility_start_date <= '2022-04-31' AND eligibility_end_date >= '2022-04-01' "
result = cursor.execute(query)
print(result.fetchone())

# Checking date stuff for roster_2
query = "SELECT * FROM roster_2 WHERE Dob >= '12/31/1930' "
result = cursor.execute(query)
print(result.fetchone())
# Conclusion: The dates in roster_2 need to be fixed

('15340006', 'Todd', 'Austin', '1934-01-06', '88', 'Male', '4731 Howe Ridge', 'California', 'New Rachel', '95451', '2021-08-01', '2023-08-01', 'Madv')
('15343508', 'Willie', 'Steele', '12/31/1939', '82', 'Male', '2058 Johnson Shoals', 'California', 'Mitchellton', '94503', '10/01/2021', '11/01/2024', 'Mdcd')


### Cleaning Inconsistent Fields Before Combining

In [7]:
# Function used to correct dates for Dob and eligibility dates in roster_2
def correct_date_format(dateString):
    numbers = dateString.split("/")
    # Changing to correct format yyyy-mm-dd
    correct_date = numbers[2] + '-' + numbers[0] + '-' + numbers[1]
    return correct_date

In [8]:
# It is confirmed that the mm/dd/yyyy format of dates doesn't work so I need to do some string manipulation
# Making changes to roster_2 using pandas:
df_roster_2 = pd.read_sql_query('select * from roster_2', connection)
df_roster_2['Dob'] = df_roster_2['Dob'].apply(lambda date: correct_date_format(date))
df_roster_2['eligibility_start_date'] = df_roster_2['eligibility_start_date'].apply(lambda date: correct_date_format(date))
df_roster_2['eligibility_end_date'] = df_roster_2['eligibility_end_date'].apply(lambda date: correct_date_format(date))
df_roster_2.head()

# Write altered dataframe to SQLite database
cursor.execute("DROP TABLE IF EXISTS roster_2_fixed")
df_roster_2.to_sql("roster_2_fixed", connection, if_exists="replace", index=False)
# NOTE: roster_2 table is locked so I have to basically rename it and stored the fixed table as a new one

In [9]:
# Trying a query from earlier to see if the dates work now
query = "SELECT * FROM roster_2_fixed WHERE eligibility_start_date <= '2022-04-31' AND eligibility_end_date >= '2022-04-01'"
result = cursor.execute(query)
print(result.fetchone())
# It works!

('15340005', 'Maritza', 'Castellana', '1979-02-09', '43', 'Female', '4097 Johnny Road', 'California', 'East Carolyntown', '93206', '2021-10-01', '2023-02-01', 'Madv')


### Combining Tables

In [10]:
# Creating table called "std_member_info" (takes a second)
cursor.execute("DROP TABLE IF EXISTS std_member_info")

# Ordering columns for EACH table because UNION considers column order, NOT column name
colNames = "Person_Id, First_Name, Last_Name, Dob, Age, Gender, Street_Address, State, City, Zip, eligibility_start_date, eligibility_end_date, payer"
r1_str = "SELECT " + colNames + " FROM roster_1"
r2_str = "SELECT " + colNames + " FROM roster_2_fixed"
r3_str = "SELECT " + colNames + " FROM roster_3"
r4_str = "SELECT " + colNames + " FROM roster_4"
r5_str = "SELECT " + colNames + " FROM roster_5"

# Executing query
query = "CREATE TABLE std_member_info AS " + r1_str + " UNION " + r2_str + " UNION " + r3_str + " UNION " + r4_str + " UNION " + r5_str
result = cursor.execute(query)

# Note 1: Logically, this could be done more efficiently by filtering BEFORE joining, but I'll keep it before filtering and dropping the columns for simplicity's sake
# Note 2: If the challenge didn't specify to 'create' a table I would have looked into creating a view so that the same data isn't stored again when creating the aggregate table

In [11]:
# Verifying we have the right number of rows
query = "SELECT COUNT(*) FROM std_member_info"
result = cursor.execute(query)
print("Num records in aggregated roster:", result.fetchone()[0])

# Getting attribute names and first record
query = "SELECT * FROM std_member_info LIMIT 1"
result = cursor.execute(query)
print("Column names:", list(map(lambda x: x[0], result.description)))
print("First record:", result.fetchone())

Num records in aggregated roster: 128530
Column names: ['Person_Id', 'First_Name', 'Last_Name', 'Dob', 'Age', 'Gender', 'Street_Address', 'State', 'City', 'Zip', 'eligibility_start_date', 'eligibility_end_date', 'payer']
First record: ('15340001', 'Daniel', 'Smith', '2017-04-27', '5', 'Male', '1505 Alvarez Spur Suite 902', 'California', 'Lake Sharonburgh', '93546', '2021-08-01', '2021-11-01', 'Madv')


### Summary Statistics for std_member_info

In [12]:
# Using pandas to easily get and print summary statistics
df = pd.read_sql_query('select * from std_member_info', connection)

In [13]:
# How many null values in the dataset?
# None :D
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128530 entries, 0 to 128529
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Person_Id               128530 non-null  object
 1   First_Name              128530 non-null  object
 2   Last_Name               128530 non-null  object
 3   Dob                     128530 non-null  object
 4   Age                     128530 non-null  object
 5   Gender                  128530 non-null  object
 6   Street_Address          128530 non-null  object
 7   State                   128530 non-null  object
 8   City                    128530 non-null  object
 9   Zip                     128530 non-null  object
 10  eligibility_start_date  128530 non-null  object
 11  eligibility_end_date    128530 non-null  object
 12  payer                   128530 non-null  object
dtypes: object(13)
memory usage: 12.7+ MB


In [14]:
# Looking at min and max values to see if we have any outliers
print(df.max(), '\n\n',df.min())

Person_Id                                  15539997
First_Name                                    Zulma
Last_Name                                    Zywiec
Dob                                      2022-06-23
Age                                              99
Gender                                         Male
Street_Address            99995 Joan Lakes Apt. 665
State                                    California
City                                    Zunigaville
Zip                                           96161
eligibility_start_date                   2022-05-01
eligibility_end_date                     2026-06-01
payer                                          Mdcd
dtype: object 

 Person_Id                           15340001
First_Name                             Aaron
Last_Name                             Aaberg
Dob                               1922-06-25
Age                                        0
Gender                                Female
Street_Address            000 Aaron M

In [15]:
# Are there inconsistencies in Gender field?
print(df.Gender.unique())

['Male' 'Female']


In [16]:
# Are there inconsistencies in the payer field?
print(df.payer.unique())

['Madv' 'Mdcd']


### Summary Statistics for model_scores_by_zip

In [17]:
# Using pandas to easily get and print summary statistics
df = pd.read_sql_query('select * from model_scores_by_zip', connection)

In [18]:
# How many null values in the dataset?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   zcta                          1760 non-null   int64  
 1   state_code                    1760 non-null   float64
 2   state name                    1760 non-null   object 
 3   neighborhood_stress_score     1760 non-null   float64
 4   algorex_sdoh_composite_score  1760 non-null   float64
 5   social_isolation_score        1760 non-null   float64
 6   transportation_access_score   1760 non-null   float64
 7   food_access_score             1760 non-null   float64
 8   unstable_housing_score        1760 non-null   float64
 9   state_govt_assistance         1760 non-null   float64
 10  homeless_indicator            1760 non-null   float64
 11  derived_indicator             1760 non-null   int64  
dtypes: float64(9), int64(2), object(1)
memory usage: 165.1+ KB


In [19]:
# Looking at min and max values to see if we have any outliers
print(df.max(), '\n\n', df.min())

zcta                                 96161
state_code                               6
state name                      California
neighborhood_stress_score             6.12
algorex_sdoh_composite_score          8.77
social_isolation_score                8.33
transportation_access_score           8.75
food_access_score                      6.9
unstable_housing_score                7.88
state_govt_assistance                  7.5
homeless_indicator                      10
derived_indicator                        1
dtype: object 

 zcta                                 90001
state_code                               6
state name                      California
neighborhood_stress_score            -1.41
algorex_sdoh_composite_score           4.3
social_isolation_score                   0
transportation_access_score           2.71
food_access_score                     0.28
unstable_housing_score                 0.2
state_govt_assistance                    0
homeless_indicator                   

### Additional Data Cleaning

In [20]:
# Aggregating by Person_Id - How many members were included more than once?
query = "SELECT Person_Id, COUNT(Person_Id) as num_duplicates FROM std_member_info GROUP BY Person_Id HAVING num_duplicates > 1"
result = cursor.execute(query)
print(len(result.fetchall()), "members were included more than once.")

10845 members were included more than once.


In [21]:
# Removing unique values
query = "DELETE FROM std_member_info WHERE rowid NOT IN (SELECT MIN(rowid) FROM std_member_info GROUP BY Person_Id)"
cursor.execute(query)

In [22]:
# This same query should now give 0
query = "SELECT Person_Id, COUNT(Person_Id) as num_duplicates FROM std_member_info GROUP BY Person_Id HAVING num_duplicates > 1"
result = cursor.execute(query)
print(len(result.fetchall()), "members were included more than once.")

0 members were included more than once.


In [23]:
# Verifying the correct number of entries are left over.
# Before we had 128530 entries
query = "SELECT COUNT(*) FROM std_member_info"
result = cursor.execute(query)
new_num = result.fetchone()[0]
# Now we have 117685
print(128530-new_num)

10845


In [24]:
# How many distinct members are eligible in April 2022?
# ASSUMPTION: I'm assuming "Eligible in April 2022" means eligible at ANY point in that month, not necessarily for the whole duration
# Ex: Eligible from 2022-04-01 to 2022-04-02 is 'eligible in April 2022'
query = "SELECT COUNT(*) FROM std_member_info WHERE eligibility_start_date <= '2022-04-31' AND eligibility_end_date >= '2022-04-01'"
result = cursor.execute(query)
print("# of distinct members eligible in April 2022:", result.fetchone()[0])

# of distinct members eligible in April 2022: 99807


In [25]:
# Dropping members that are not eligible in April 2022
query = "DELETE FROM std_member_info WHERE Person_Id NOT IN (SELECT Person_Id FROM std_member_info WHERE eligibility_start_date <= '2022-04-31' AND eligibility_end_date >= '2022-04-01')"
cursor.execute(query)

# Confirming that there are the correct # of records
query = "SELECT COUNT(*) FROM std_member_info"
result = cursor.execute(query)
print(result.fetchone())

(99807,)


In [26]:
# Now we can drop the two eligibility columns (as instructed by challenge)
# SQLite doesn't support ALTER TABLE [] DROP, so I'm going to just drop them using pandas
df = pd.read_sql_query('select Person_Id, First_Name, Last_Name, Dob, Age, Gender, Street_Address, State, City, Zip, payer from std_member_info', connection)
query = "DROP TABLE std_member_info"
cursor.execute(query)
df.to_sql("std_member_info", connection, if_exists="replace", index=False)

In [27]:
# Using pandas (again) to confirm that the updated std_member_info attributes are stored in the DB
df = pd.read_sql_query('select * from std_member_info', connection)
df.head()

,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,payer
0,15340003,Todd,Morelli,1993-06-29,29,Male,4070 Aguilar Forge Suite 788,California,South Melissaton,92105,Madv
1,15340005,Maritza,Castellana,1979-02-09,43,Female,4097 Johnny Road,California,East Carolyntown,93206,Madv
2,15340006,Todd,Austin,1934-01-06,88,Male,4731 Howe Ridge,California,New Rachel,95451,Madv
3,15340009,Florence,Kirk,2016-05-06,6,Female,31252 Wade Inlet,California,North Toddborough,93616,Mdcd
4,15340010,Jimmy,Clark,1926-07-16,96,Male,9763 Cynthia Vista Suite 910,California,Lake Morgan,96096,Mdcd


### Answering Questions (post-cleaning)

In [28]:
# What is the breakdown of members by payer?
query = "SELECT payer, AVG(age) as avg_age, COUNT(*) as num_members FROM std_member_info GROUP BY payer"
result = cursor.execute(query)
print(result.fetchall())
# Madv has 37352 members with an average age of ~50 years, and Mdcd has 62455 members, also with an average age of ~50 years.

[('Madv', 49.90003212679375, 37352), ('Mdcd', 49.89751020734929, 62455)]


In [29]:
# How many members live in a zip code with a food_access_score lower than 2?
query = "SELECT COUNT(*) FROM std_member_info m JOIN model_scores_by_zip z ON m.Zip = z.zcta WHERE food_access_score < 2"
result = cursor.execute(query)
print(result.fetchall())
# There are 7,685 members in a zip code with a food_access_score lower than 2

[(7685,)]


In [30]:
# What is the average social isolation score for the members?
query = "SELECT AVG(social_isolation_score) FROM std_member_info m JOIN model_scores_by_zip z ON m.Zip = z.zcta"
result = cursor.execute(query)
print(result.fetchall())
# The average social isolation score is 3.07

[(3.068896570380987,)]


In [31]:
# Which members live in the zip code with the highest algorex_sdoh_composite_score?

# Which zip code has the highest algorex_sdoh_composite_score?
query = "SELECT zcta FROM model_scores_by_zip ORDER BY algorex_sdoh_composite_score DESC LIMIT 1"
result = cursor.execute(query)
print("Highest zip:", result.fetchall()[0])
# Zip code 95950 has the highest score, which is 8.77

# This is the table from the query above renamed
highest_ascore_table = "(SELECT zcta FROM model_scores_by_zip ORDER BY algorex_sdoh_composite_score DESC LIMIT 1)"
query = "SELECT Person_Id FROM std_member_info m JOIN model_scores_by_zip z ON m.Zip = z.zcta WHERE zcta IN " + highest_ascore_table
result = cursor.execute(query)

# Getting it into a nice list
members_raw = result.fetchall()
members = []
for tuple in members_raw[:]:
    members.append(tuple[0])
print("List of members:", members)

Highest zip: (95950,)
List of members: ['15340102', '15346807', '15348534', '15355184', '15359217', '15363143', '15370952', '15375855', '15377622', '15377751', '15383576', '15387928', '15389381', '15389759', '15403845', '15404143', '15414827', '15447186', '15456488', '15456560', '15459547', '15460500', '15462579', '15468254', '15469273', '15476293', '15482255', '15484118', '15484216', '15486908', '15487938', '15489640', '15493624', '15495072', '15499581', '15502702', '15506131', '15514403', '15516212', '15525968', '15531858', '15532243', '15534578', '15537550', '15539805']


### Cleaning it up

In [32]:
cursor.execute("DROP TABLE IF EXISTS roster_2_fixed")
cursor.close()
connection.close()